# Setup

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle

# Preprocessing

## Load 1 Hz data

In [2]:
hz_dataset_path = 'data/WEEE Dataset/processed/merged_weee_dataset.csv'
hz_df = pd.read_csv(hz_dataset_path, parse_dates=['Time']).sort_values(by='Time')
hz_df

,Time,heart_rate,VO2[mL/kg/min],VO2[mL/min],t,Weight,Height,Gender,Age,subject_id,EE[kcal/min],EE[kcal/kg/min]
1799,2021-11-25 17:17:43,64.765699,2.52,188.67,0,77.0,173,1,26,P02,930.89778,12.43368
1800,2021-11-25 17:17:44,59.656709,2.52,188.67,1,77.0,173,1,26,P02,930.89778,12.43368
1801,2021-11-25 17:17:45,110.264730,2.52,188.67,2,77.0,173,1,26,P02,930.89778,12.43368
1802,2021-11-25 17:17:46,84.384969,2.52,188.67,3,77.0,173,1,26,P02,930.89778,12.43368
1803,2021-11-25 17:17:47,60.196520,2.34,175.33,4,77.0,173,1,26,P02,865.07822,11.54556
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,2021-12-03 17:28:47,134.920374,13.34,920.33,1794,70.6,166,0,29,P01,4540.90822,65.81956
1795,2021-12-03 17:28:48,159.813979,13.34,920.33,1795,70.6,166,0,29,P01,4540.90822,65.81956
1796,2021-12-03 17:28:49,140.897191,13.34,920.33,1796,70.6,166,0,29,P01,4540.90822,65.81956
1797,2021-12-03 17:28:50,154.094574,13.34,920.33,1797,70.6,166,0,29,P01,4540.90822,65.81956


In [3]:
hz_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27895 entries, 1799 to 1798
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Time             27895 non-null  datetime64[ns]
 1   heart_rate       27895 non-null  float64       
 2   VO2[mL/kg/min]   27895 non-null  float64       
 3   VO2[mL/min]      27895 non-null  float64       
 4   t                27895 non-null  int64         
 5   Weight           27895 non-null  float64       
 6   Height           27895 non-null  int64         
 7   Gender           27895 non-null  int64         
 8   Age              27895 non-null  int64         
 9   subject_id       27895 non-null  object        
 10  EE[kcal/min]     27895 non-null  float64       
 11  EE[kcal/kg/min]  27895 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 2.8+ MB


## Load raw acceleration data

In [4]:
filtered_acc_path = 'data/WEEE Dataset/processed/filtered_acceleration.csv'
filtered_acc_df = pd.read_csv(filtered_acc_path, parse_dates=['Time']).sort_values(by='Time')
filtered_acc_df

,X,Y,Z,Time,subject_id
57568,42.0,-29.0,36.0,2021-11-25 17:17:43.000000,P02
57569,42.0,-29.0,37.0,2021-11-25 17:17:43.031250,P02
57570,41.0,-29.0,37.0,2021-11-25 17:17:43.062500,P02
57571,42.0,-29.0,37.0,2021-11-25 17:17:43.093750,P02
57572,42.0,-29.0,36.0,2021-11-25 17:17:43.125000,P02
...,...,...,...,...,...
57563,12.0,-48.0,30.0,2021-12-03 17:28:51.843750,P01
57564,21.0,-48.0,43.0,2021-12-03 17:28:51.875000,P01
57565,17.0,-49.0,35.0,2021-12-03 17:28:51.906250,P01
57566,21.0,-51.0,34.0,2021-12-03 17:28:51.937500,P01


In [5]:
filtered_acc_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 892640 entries, 57568 to 57567
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   X           892640 non-null  float64       
 1   Y           892640 non-null  float64       
 2   Z           892640 non-null  float64       
 3   Time        892640 non-null  datetime64[ns]
 4   subject_id  892640 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 40.9+ MB


## Merge data

In [6]:
merged_df = pd.merge_asof(
    filtered_acc_df, hz_df,
    on='Time',
    by='subject_id',
    direction='backward'
).sort_values(by=['subject_id','Time']).reset_index(drop=True)
merged_df

,X,Y,Z,Time,subject_id,heart_rate,VO2[mL/kg/min],VO2[mL/min],t,Weight,Height,Gender,Age,EE[kcal/min],EE[kcal/kg/min]
0,29.0,0.0,56.0,2021-12-03 16:58:53.000000,P01,94.690403,6.81,470.17,0,70.6,166,0,29,2319.81878,33.60054
1,29.0,0.0,56.0,2021-12-03 16:58:53.031250,P01,94.690403,6.81,470.17,0,70.6,166,0,29,2319.81878,33.60054
2,29.0,0.0,55.0,2021-12-03 16:58:53.062500,P01,94.690403,6.81,470.17,0,70.6,166,0,29,2319.81878,33.60054
3,30.0,0.0,56.0,2021-12-03 16:58:53.093750,P01,94.690403,6.81,470.17,0,70.6,166,0,29,2319.81878,33.60054
4,30.0,0.0,56.0,2021-12-03 16:58:53.125000,P01,94.690403,6.81,470.17,0,70.6,166,0,29,2319.81878,33.60054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892635,7.0,-47.0,-9.0,2021-12-02 17:34:07.843750,P17,100.206843,11.12,858.83,1798,77.2,175,1,33,4237.46722,54.86608
892636,-24.0,-71.0,-15.0,2021-12-02 17:34:07.875000,P17,100.206843,11.12,858.83,1798,77.2,175,1,33,4237.46722,54.86608
892637,-31.0,-63.0,-8.0,2021-12-02 17:34:07.906250,P17,100.206843,11.12,858.83,1798,77.2,175,1,33,4237.46722,54.86608
892638,-25.0,-56.0,-7.0,2021-12-02 17:34:07.937500,P17,100.206843,11.12,858.83,1798,77.2,175,1,33,4237.46722,54.86608


### Drop extra columns

In [7]:
keep_cols = [
    # time and subject id
    'Time',
    'subject_id',
    # accelerometer
    'X',
    'Y',
    'Z',
    # heart rate
    'heart_rate',
    # demographics
    'Weight',
    'Height',
    'Gender',
    'Age',
    # label
    'EE[kcal/min]'

]
merged_df_smol = merged_df[keep_cols]
merged_df_smol

,Time,subject_id,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,2021-12-03 16:58:53.000000,P01,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
1,2021-12-03 16:58:53.031250,P01,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
2,2021-12-03 16:58:53.062500,P01,29.0,0.0,55.0,94.690403,70.6,166,0,29,2319.81878
3,2021-12-03 16:58:53.093750,P01,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
4,2021-12-03 16:58:53.125000,P01,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
...,...,...,...,...,...,...,...,...,...,...,...
892635,2021-12-02 17:34:07.843750,P17,7.0,-47.0,-9.0,100.206843,77.2,175,1,33,4237.46722
892636,2021-12-02 17:34:07.875000,P17,-24.0,-71.0,-15.0,100.206843,77.2,175,1,33,4237.46722
892637,2021-12-02 17:34:07.906250,P17,-31.0,-63.0,-8.0,100.206843,77.2,175,1,33,4237.46722
892638,2021-12-02 17:34:07.937500,P17,-25.0,-56.0,-7.0,100.206843,77.2,175,1,33,4237.46722


In [8]:
merged_df_smol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892640 entries, 0 to 892639
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Time          892640 non-null  datetime64[ns]
 1   subject_id    892640 non-null  object        
 2   X             892640 non-null  float64       
 3   Y             892640 non-null  float64       
 4   Z             892640 non-null  float64       
 5   heart_rate    892640 non-null  float64       
 6   Weight        892640 non-null  float64       
 7   Height        892640 non-null  int64         
 8   Gender        892640 non-null  int64         
 9   Age           892640 non-null  int64         
 10  EE[kcal/min]  892640 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 74.9+ MB


In [9]:
merged_df_smol[merged_df_smol['subject_id'] == 'P01']

,Time,subject_id,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,2021-12-03 16:58:53.000000,P01,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
1,2021-12-03 16:58:53.031250,P01,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
2,2021-12-03 16:58:53.062500,P01,29.0,0.0,55.0,94.690403,70.6,166,0,29,2319.81878
3,2021-12-03 16:58:53.093750,P01,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
4,2021-12-03 16:58:53.125000,P01,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
...,...,...,...,...,...,...,...,...,...,...,...
57563,2021-12-03 17:28:51.843750,P01,12.0,-48.0,30.0,142.848414,70.6,166,0,29,6372.26100
57564,2021-12-03 17:28:51.875000,P01,21.0,-48.0,43.0,142.848414,70.6,166,0,29,6372.26100
57565,2021-12-03 17:28:51.906250,P01,17.0,-49.0,35.0,142.848414,70.6,166,0,29,6372.26100
57566,2021-12-03 17:28:51.937500,P01,21.0,-51.0,34.0,142.848414,70.6,166,0,29,6372.26100


## Standardize features (skip since moving to new file)

In [10]:
feature_cols = [
    # accelerometer
    'X',
    'Y',
    'Z',
    # heart rate
    'heart_rate',
    # demographics
    'Weight',
    'Height',
    # 'Gender',
    'Age',
    # label
    'EE[kcal/min]'
]
scaler = StandardScaler()
std_df = merged_df_smol.copy()
std_df[feature_cols] = scaler.fit_transform(std_df[feature_cols])

In [11]:
std_df

,Time,subject_id,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,2021-12-03 16:58:53.000000,P01,0.220972,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
1,2021-12-03 16:58:53.031250,P01,0.220972,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
2,2021-12-03 16:58:53.062500,P01,0.220972,0.454626,1.252592,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
3,2021-12-03 16:58:53.093750,P01,0.249426,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
4,2021-12-03 16:58:53.125000,P01,0.249426,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
...,...,...,...,...,...,...,...,...,...,...,...
892635,2021-12-02 17:34:07.843750,P17,-0.405016,-0.806902,-1.415019,0.640514,0.348866,0.405931,1,0.531476,0.274632
892636,2021-12-02 17:34:07.875000,P17,-1.287090,-1.451086,-1.665107,0.640514,0.348866,0.405931,1,0.531476,0.274632
892637,2021-12-02 17:34:07.906250,P17,-1.486268,-1.236358,-1.373337,0.640514,0.348866,0.405931,1,0.531476,0.274632
892638,2021-12-02 17:34:07.937500,P17,-1.315544,-1.048471,-1.331656,0.640514,0.348866,0.405931,1,0.531476,0.274632


In [12]:
std_df.describe()

,Time,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
count,892640,8.926400e+05,8.926400e+05,8.926400e+05,8.926400e+05,8.926400e+05,8.926400e+05,892640.000000,8.926400e+05,8.926400e+05
mean,2021-11-30 05:52:56.382816,6.113291e-18,6.317068e-17,-2.852869e-17,-1.221639e-15,-1.244564e-15,-1.752477e-15,0.721742,-5.909515e-17,-4.849878e-16
min,2021-11-25 17:17:43,-4.246308e+00,-2.981023e+00,-6.375107e+00,-2.695758e+00,-1.774044e+00,-1.877081e+00,0.000000,-1.391770e+00,-1.107577e+00
25%,2021-11-29 10:42:23.742187520,-5.757401e-01,-6.995375e-01,-7.481159e-01,-5.311926e-01,-6.459706e-01,-9.638765e-01,0.000000,-8.147960e-01,-8.532939e-01
50%,2021-11-30 11:45:17.484375040,2.179417e-02,-3.237635e-01,1.688752e-01,4.038343e-02,-1.307874e-01,1.015290e-01,1.000000,-2.378220e-01,-3.148776e-01
75%,2021-12-01 15:32:36.226562560,5.624204e-01,7.230357e-01,8.774593e-01,5.749552e-01,3.488660e-01,7.103322e-01,1.000000,5.314765e-01,5.779115e-01
max,2021-12-03 17:28:51.968750,3.009465e+00,3.863433e+00,4.253654e+00,3.007663e+00,2.347422e+00,2.080139e+00,1.000000,2.070074e+00,4.879308e+00
std,NaN,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,0.448141,1.000001e+00,1.000001e+00


## Apply sliding window

In [13]:
def create_sliding_windows(df, window_size=192, overlap=0.5, sample_rate=32) -> dict[list[pd.DataFrame]]:
    windows = {}
    step_size = int(window_size * overlap)
    max_gap_seconds = window_size / sample_rate

    for subject_id, group in df.groupby('subject_id'):
        windows[subject_id] = []

        group = group.sort_values(by='Time').reset_index(drop=True)
        group['time_diff'] = group['Time'].diff().dt.total_seconds()

        valid_sections = (group['time_diff'] <= max_gap_seconds) | (group['time_diff'].isna())
        group['valid_sections'] = valid_sections

        group['section_id'] = (~valid_sections).astype(int).cumsum()

        # print(f"{subject_id} diffs: {group[['Time', 'time_diff']].head(20)}")
        # print(f"Section ids count: {group['section_id'].value_counts()}")
        # print(group[['Time', 'time_diff', 'valid_sections']][~group['valid_sections']])

        for section_id, section in group.groupby('section_id'):
            if len(section) < window_size:
                # print(f"Skipping section {section_id} for subject {subject_id} (length {len(section)})")
                continue

            # print(f"{section_id}: {group[['Time', 'time_diff', 'section_id']].head(20)}")

            section.drop(columns=['time_diff', 'valid_sections'], inplace=True) #, 'section_id'

            for start in range(0, len(section) - window_size + 1, step_size):
                windows[subject_id].append(section.iloc[start:start + window_size].copy().reset_index(drop=True))

    return windows

In [24]:
# windows_dict = create_sliding_windows(std_df)
windows_dict = create_sliding_windows(merged_df_smol)
windows_dict.keys()

dict_keys(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17'])

In [25]:
len(windows_dict['P01'])

598

In [26]:
def display_windows(subject_windows_list):
        display(subject_windows_list[0])
        display(subject_windows_list[-1])

In [27]:
display_windows(windows_dict['P01'])

,Time,subject_id,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min],section_id
0,2021-12-03 16:58:53.000000,P01,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878,0
1,2021-12-03 16:58:53.031250,P01,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878,0
2,2021-12-03 16:58:53.062500,P01,29.0,0.0,55.0,94.690403,70.6,166,0,29,2319.81878,0
3,2021-12-03 16:58:53.093750,P01,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878,0
4,2021-12-03 16:58:53.125000,P01,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878,0
...,...,...,...,...,...,...,...,...,...,...,...,...
187,2021-12-03 16:58:58.843750,P01,30.0,0.0,55.0,94.515007,70.6,166,0,29,2424.22222,0
188,2021-12-03 16:58:58.875000,P01,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222,0
189,2021-12-03 16:58:58.906250,P01,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222,0
190,2021-12-03 16:58:58.937500,P01,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222,0


,Time,subject_id,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min],section_id
0,2021-12-03 17:28:44.000000,P01,19.0,-46.0,44.0,151.930727,70.6,166,0,29,5812.25200,0
1,2021-12-03 17:28:44.031250,P01,14.0,-47.0,38.0,151.930727,70.6,166,0,29,5812.25200,0
2,2021-12-03 17:28:44.062500,P01,17.0,-46.0,46.0,151.930727,70.6,166,0,29,5812.25200,0
3,2021-12-03 17:28:44.093750,P01,12.0,-49.0,38.0,151.930727,70.6,166,0,29,5812.25200,0
4,2021-12-03 17:28:44.125000,P01,21.0,-50.0,33.0,151.930727,70.6,166,0,29,5812.25200,0
...,...,...,...,...,...,...,...,...,...,...,...,...
187,2021-12-03 17:28:49.843750,P01,8.0,-47.0,40.0,140.897191,70.6,166,0,29,4540.90822,0
188,2021-12-03 17:28:49.875000,P01,22.0,-48.0,41.0,140.897191,70.6,166,0,29,4540.90822,0
189,2021-12-03 17:28:49.906250,P01,15.0,-53.0,29.0,140.897191,70.6,166,0,29,4540.90822,0
190,2021-12-03 17:28:49.937500,P01,23.0,-52.0,34.0,140.897191,70.6,166,0,29,4540.90822,0


In [28]:
display_windows(windows_dict['P16'])

,Time,subject_id,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min],section_id
0,2021-12-01 18:44:37.000000,P16,-50.0,21.0,35.0,22.496484,73.1,172,1,34,1021.338,0
1,2021-12-01 18:44:37.031250,P16,-49.0,21.0,35.0,22.496484,73.1,172,1,34,1021.338,0
2,2021-12-01 18:44:37.062500,P16,-49.0,20.0,35.0,22.496484,73.1,172,1,34,1021.338,0
3,2021-12-01 18:44:37.093750,P16,-49.0,20.0,36.0,22.496484,73.1,172,1,34,1021.338,0
4,2021-12-01 18:44:37.125000,P16,-49.0,20.0,36.0,22.496484,73.1,172,1,34,1021.338,0
...,...,...,...,...,...,...,...,...,...,...,...,...
187,2021-12-01 18:44:42.843750,P16,-46.0,19.0,40.0,122.344096,73.1,172,1,34,1028.739,0
188,2021-12-01 18:44:42.875000,P16,-46.0,19.0,40.0,122.344096,73.1,172,1,34,1028.739,0
189,2021-12-01 18:44:42.906250,P16,-46.0,19.0,41.0,122.344096,73.1,172,1,34,1028.739,0
190,2021-12-01 18:44:42.937500,P16,-46.0,19.0,40.0,122.344096,73.1,172,1,34,1028.739,0


,Time,subject_id,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min],section_id
0,2021-12-01 19:22:49.000000,P16,-71.0,55.0,44.0,102.565465,73.1,172,1,34,8577.759,3
1,2021-12-01 19:22:49.031250,P16,-16.0,57.0,60.0,102.565465,73.1,172,1,34,8577.759,3
2,2021-12-01 19:22:49.062500,P16,33.0,28.0,33.0,102.565465,73.1,172,1,34,8577.759,3
3,2021-12-01 19:22:49.093750,P16,41.0,32.0,15.0,102.565465,73.1,172,1,34,8577.759,3
4,2021-12-01 19:22:49.125000,P16,37.0,49.0,20.0,102.565465,73.1,172,1,34,8577.759,3
...,...,...,...,...,...,...,...,...,...,...,...,...
187,2021-12-01 19:22:54.843750,P16,10.0,118.0,6.0,87.827431,73.1,172,1,34,8464.277,3
188,2021-12-01 19:22:54.875000,P16,-112.0,127.0,30.0,87.827431,73.1,172,1,34,8464.277,3
189,2021-12-01 19:22:54.906250,P16,-128.0,124.0,39.0,87.827431,73.1,172,1,34,8464.277,3
190,2021-12-01 19:22:54.937500,P16,-128.0,79.0,44.0,87.827431,73.1,172,1,34,8464.277,3


## Remove extra columns

In [29]:
drop_cols = [
    'Time',
    'subject_id',
    'section_id'
]
for subject_window_list in windows_dict.values():
    for window in subject_window_list:
        window.drop(columns=drop_cols, inplace=True)

In [30]:
display_windows(windows_dict['P01'])

,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
1,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
2,29.0,0.0,55.0,94.690403,70.6,166,0,29,2319.81878
3,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
4,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
...,...,...,...,...,...,...,...,...,...
187,30.0,0.0,55.0,94.515007,70.6,166,0,29,2424.22222
188,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222
189,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222
190,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222


,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,19.0,-46.0,44.0,151.930727,70.6,166,0,29,5812.25200
1,14.0,-47.0,38.0,151.930727,70.6,166,0,29,5812.25200
2,17.0,-46.0,46.0,151.930727,70.6,166,0,29,5812.25200
3,12.0,-49.0,38.0,151.930727,70.6,166,0,29,5812.25200
4,21.0,-50.0,33.0,151.930727,70.6,166,0,29,5812.25200
...,...,...,...,...,...,...,...,...,...
187,8.0,-47.0,40.0,140.897191,70.6,166,0,29,4540.90822
188,22.0,-48.0,41.0,140.897191,70.6,166,0,29,4540.90822
189,15.0,-53.0,29.0,140.897191,70.6,166,0,29,4540.90822
190,23.0,-52.0,34.0,140.897191,70.6,166,0,29,4540.90822


## Export Windows

In [31]:
windows_path = 'data/WEEE Dataset/processed/windows.pkl'
with open(windows_path, 'wb') as f:
    pickle.dump(windows_dict, f)

## Check export

In [32]:
with open(windows_path, 'rb') as f:
    windows_dict = pickle.load(f)

windows_dict.keys()

dict_keys(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17'])

In [33]:
display_windows(windows_dict['P01'])

,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
1,29.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
2,29.0,0.0,55.0,94.690403,70.6,166,0,29,2319.81878
3,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
4,30.0,0.0,56.0,94.690403,70.6,166,0,29,2319.81878
...,...,...,...,...,...,...,...,...,...
187,30.0,0.0,55.0,94.515007,70.6,166,0,29,2424.22222
188,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222
189,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222
190,30.0,0.0,56.0,94.515007,70.6,166,0,29,2424.22222


,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,19.0,-46.0,44.0,151.930727,70.6,166,0,29,5812.25200
1,14.0,-47.0,38.0,151.930727,70.6,166,0,29,5812.25200
2,17.0,-46.0,46.0,151.930727,70.6,166,0,29,5812.25200
3,12.0,-49.0,38.0,151.930727,70.6,166,0,29,5812.25200
4,21.0,-50.0,33.0,151.930727,70.6,166,0,29,5812.25200
...,...,...,...,...,...,...,...,...,...
187,8.0,-47.0,40.0,140.897191,70.6,166,0,29,4540.90822
188,22.0,-48.0,41.0,140.897191,70.6,166,0,29,4540.90822
189,15.0,-53.0,29.0,140.897191,70.6,166,0,29,4540.90822
190,23.0,-52.0,34.0,140.897191,70.6,166,0,29,4540.90822
